In [30]:
# Can't fine tune the model on my laptop, not enough VRAM
!nvidia-smi

Sat Jul 15 11:08:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.40       Driver Version: 516.40       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8     3W /  N/A |   3881MiB /  4096MiB |      7%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

c:\Users\siddp\OneDrive\Desktop\project-venv\transformer-piepline\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\siddp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
torch.cuda.empty_cache()

In [3]:
# Usualy I directly use .cuda()

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
device

'cuda'

In [4]:
# This is a text summarization transformer model on HuggingFace by google.

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [5]:
model_pegasus.config.output_attentions

False

In [6]:
# !pip install zipfile36
# import zipfile

In [7]:
#dowload & unzip data

# !wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
# !python -m zipfile -e summarizer-data.zip data/

In [8]:
dataset_samsum = load_from_disk('data/samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
dataset_samsum["test"][1]

{'id': '13729565',
 'dialogue': "Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric: I know! And shows how Americans see Russian ;)\r\nRob: And it's really funny!\r\nEric: I know! I especially like the train part!\r\nRob: Hahaha! No one talks to the machine like that!\r\nEric: Is this his only stand-up?\r\nRob: Idk. I'll check.\r\nEric: Sure.\r\nRob: Turns out no! There are some of his stand-ups on youtube.\r\nEric: Gr8! I'll watch them now!\r\nRob: Me too!\r\nEric: MACHINE!\r\nRob: MACHINE!\r\nEric: TTYL?\r\nRob: Sure :)",
 'summary': 'Eric and Rob are going to watch a stand-up on youtube.'}

In [10]:
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")
print(dataset_samsum["test"][1]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][1]["summary"])

Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [11]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length=1024, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)
        
    return {
        "ie": input_encodings,
        "te": target_encodings
    }

In [12]:
example = convert_examples_to_features(dataset_samsum['train'][0])

example["ie"].keys()

c:\Users\siddp\OneDrive\Desktop\project-venv\transformer-piepline\lib\site-packages\transformers\tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


dict_keys(['input_ids', 'attention_mask'])

In [13]:
print(example["ie"]['input_ids'])
print(example["ie"]['attention_mask'])

[12195, 151, 125, 7091, 3659, 107, 842, 119, 245, 181, 152, 10508, 151, 7435, 147, 12195, 151, 125, 131, 267, 650, 119, 3469, 29344, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [14]:
example["te"].keys()

dict_keys(['input_ids', 'attention_mask'])

In [15]:
print(example["te"]['input_ids'])
print(example["te"]['attention_mask'])

[12195, 7091, 3659, 111, 138, 650, 10508, 181, 3469, 107, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [16]:
# input_ids (torch.LongTensor of shape (batch_size, sequence_length)) — 
# Indices of input sequence tokens in the vocabulary. Padding will be 
# ignored by default should you provide it.

# attention_mask (torch.Tensor of shape (batch_size, sequence_length), 
# optional) — Mask to avoid performing attention on padding token 
# indices. Mask values selected in [0, 1], 0 for masking.

# decoder_input_ids (torch.LongTensor of shape 
# (batch_size, target_sequence_length), optional) — Indices of 
# decoder input sequence tokens in the vocabulary.

In [17]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 256, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [18]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,
                                        batched=True)

Loading cached processed dataset at c:\Users\siddp\OneDrive\Desktop\project\end-to-end pipeline for text summarization\text-summarization-transformer\research\data\samsum_dataset\train\cache-f68ef63b7e6f98cd.arrow
Loading cached processed dataset at c:\Users\siddp\OneDrive\Desktop\project\end-to-end pipeline for text summarization\text-summarization-transformer\research\data\samsum_dataset\test\cache-64834577e7897630.arrow
Loading cached processed dataset at c:\Users\siddp\OneDrive\Desktop\project\end-to-end pipeline for text summarization\text-summarization-transformer\research\data\samsum_dataset\validation\cache-976bbac2fc24c48d.arrow


In [19]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [20]:
# Training The model

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, 
                                               model=model_pegasus)

In [21]:
# !pip install accelerate -U

In [26]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 

In [27]:
# Model to big, so getting information of GPU capacity to figure out
# better batch sizes and other parameters

# print(torch.cuda.memory_summary(device=device))

In [28]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["validation"], 
                  eval_dataset=dataset_samsum_pt["test"])

In [29]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 3.46 GiB already allocated; 0 bytes free; 3.48 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF